In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [22]:
base_url = "https://www.creditkarma.com/credit-cards/search-cc"
response = requests.get(base_url)

soup = BeautifulSoup(response.content, 'html.parser')
cards = soup.find(id="anchor")
print(cards.prettify())


<div id="anchor">
 <div class="jsx-3196776952 marketplace min-vh-100 pb3 bg-white">
  <div class="jsx-1132646303">
   <div class="jsx-1132646303">
    <nav class="jsx-1812221928 bg-white ck-shadow pv3">
     <div class="jsx-1812221928 b ph3 ph0-l mw8 center pb2">
      Browse categories
     </div>
     <div class="jsx-1812221928 mw8 center">
      <div class="jsx-661246960 carousel-lite relative">
       <div class="jsx-661246960 scroller-container overflow-hidden">
        <div class="jsx-661246960 scroller overflow-hidden-ns overflow-x-scroll flex flex-nowrap ph0 pv0">
         <div class="outer block relative overflow-y-hidden">
          <div class="middle block overflow-x-auto relative nb4">
           <div class="inner inline-flex pr3 ph3 ph1-ns pb4">
            <a class="flex items-center ck-black-80 pointer f5 hover-bg-ck-primary-10 pill-nav-item ck-height-s-element ph2 mr2 ba justify-center b--ck-black-50 br2" href="/credit-cards">
             <span class="category-name">
 

In [23]:
credit_cards = cards.find_all("div", class_="jsx-2570701536")

for card in credit_cards:
    card_name = card.find("h2", class_="lh-copy mv0 f4 f3-ns")

    details = card.find("li", class_="flex flex-column bb b--light-gray pv2 ph3 w-50")
    if details:
        details_text = details.text.strip() 

        if "Cash back" in details_text or "Miles per dollar" in details_text:
            cashback_match = re.search(r'(\d+(?:\.\d+)?%)\s*[-–]\s*(\d+(?:\.\d+)?%)|(\d+(?:\.\d+)?%)', details_text)
            if cashback_match:
                if cashback_match.group(1) and cashback_match.group(2):
                    details_re = f"Cash back: {cashback_match.group(1)} - {cashback_match.group(2)}"
                elif cashback_match.group(3):
                    details_re = f"Cash back: {cashback_match.group(3)}"
            else:
                details_re = "No cash back"
        elif "APR" in details_text:
            apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*-\s*(\d+(?:\.\d+)?%))?', details_text)
            if apr_match:
                details_re = "APR: " + apr_match.group(1)
            else:
                details_re = "No APR"
        else:
            details_re = "No APR or cash back"
    

        annual_fee = card.find("li", class_="flex flex-column bb b--light-gray pv2 ph3 w-50 bl")
        annual_fee_text = annual_fee.text.strip() if annual_fee else "$0"
        annual_fee = re.search(r'\$\d+', annual_fee_text)
        if annual_fee:
            annual_fee = annual_fee.group()
        else:
            annual_fee = "$0"

        welcome_bonus = "$0"
    else:
        details = card.find("li", class_="flex flex-column bb b--light-gray pv2 ph3 ckoffer-w-third")
        details_text = details.text.strip() if details else "$0"

        if "Cash back" in details_text or "Miles per dollar" in details_text:
            cashback_match = re.search(r'(\d+(?:\.\d+)?%)\s*[-–]\s*(\d+(?:\.\d+)?%)|(\d+(?:\.\d+)?%)', details_text)
            if cashback_match:
                if cashback_match.group(1) and cashback_match.group(2):
                    details_re = f"Cash back: {cashback_match.group(1)} - {cashback_match.group(2)}"
                elif cashback_match.group(3):
                    details_re = f"Cash back: {cashback_match.group(3)}"
            else:
                details_re = "No cash back"
        elif "APR" in details_text:
            apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*-\s*(\d+(?:\.\d+)?%))?', details_text)
            if apr_match:
                details_re = "APR: " + apr_match.group(1)
            else:
                details_re = "No APR"
        else:
            details_re = "No APR or cash back"
        
        annual_fee = "$0"
        welcome_bonus = "$0"

        annual_or_bonus = card.find_all("li", class_="flex flex-column bb b--light-gray pv2 ph3 ckoffer-w-third bl")
        for item in annual_or_bonus:
            item_text = item.text.strip()

            if "Annual Fee" in item_text:
                annual_fee_match = re.search(r'\$\d+', item_text)
                if annual_fee_match:
                    annual_fee = annual_fee_match.group()

            if "Welcome" in item_text or "Bonus" in item_text or "Intro offer" in item_text:
                bonus_match = re.search(r'\$\d+[,\d]*', item_text) 
                if bonus_match:
                    welcome_bonus = bonus_match.group()

            
    
    print("Name:" ,card_name.text.strip())
    print(details_re)
    print("Annual Fee:" ,annual_fee) 
    print("Welcome Bonus:" ,welcome_bonus)
    print("")
    
   
   

Name: Credit One Bank® Platinum X5 Visa®
Cash back: 1% - 5%
Annual Fee: $95
Welcome Bonus: $0

Name: Capital One Quicksilver Cash Rewards Credit Card
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $200

Name: Upgrade Cash Rewards Visa®
Cash back: 1.5%
Annual Fee: $0
Welcome Bonus: $0

Name: Capital One VentureOne Rewards for Good Credit
No cash back
Annual Fee: $0
Welcome Bonus: $0

Name: First Latitude Assent Mastercard® Secured Credit Card
No APR or cash back
Annual Fee: $0
Welcome Bonus: $0

Name: Mission Lane Visa® Credit Card
APR: 33.99%
Annual Fee: $39
Welcome Bonus: $0

Name: Capital One Quicksilver Cash Rewards for Good Credit
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $0

Name: Wells Fargo Active Cash® Card
Cash back: 2%
Annual Fee: $0
Welcome Bonus: $200

Name: Milestone® Mastercard® - With A Higher Credit Limit
APR: 35.90%
Annual Fee: $125
Welcome Bonus: $0

Name: Credit One Bank® Platinum Rewards Visa®
Cash back: 1%
Annual Fee: $0
Welcome Bonus: $0

Name: Milest

In [24]:
credit_cards = cards.find_all("div", class_="jsx-2570701536")

for card in credit_cards:
    card_name_tag = card.find("h2", class_="lh-copy mv0 f4 f3-ns")
    card_name = card_name_tag.text.strip() if card_name_tag else "No name"

    cashback = "No cash back"
    apr = "No APR"
    annual_fee = "$0"
    welcome_bonus = "$0"
    cashback = "No cash back"
    points = "No points"

    # Get all info boxes
    info_boxes = card.find_all("li")
    
    for item in info_boxes[:4]:
        text = item.text.strip()

        # Cash back or Miles
        if "Cash back" in text or "Miles per dollar" in text or "Rewards rate" in text:
            rewards_match = re.search(
            r'(\d+(?:\.\d+)?[x%])\s*[-–]\s*(\d+(?:\.\d+)?[x%])|(\d+(?:\.\d+)?[x%])', text, re.IGNORECASE)

        if rewards_match:
            if rewards_match.group(1) and rewards_match.group(2):
                first_unit = rewards_match.group(1)[-1]
                if first_unit == '%':
                    cashback = f"Cash back: {rewards_match.group(1)} - {rewards_match.group(2)}"
                elif first_unit.lower() == 'x':
                    points = f"Points: {rewards_match.group(1)} - {rewards_match.group(2)}"
            elif rewards_match.group(3):
                single_unit = rewards_match.group(3)[-1]
                if single_unit == '%':
                    cashback = f"Cash back: {rewards_match.group(3)}"
                elif single_unit.lower() == 'x':
                    points = f"Points: {rewards_match.group(3)}"

        # APR
        if "APR" in text:
            apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*[-–]\s*(\d+(?:\.\d+)?%))?', text)
            if apr_match:
                if apr_match.group(2):
                    apr = f"APR: {apr_match.group(1)} - {apr_match.group(2)}"
                else:
                    apr = f"APR: {apr_match.group(1)}"

        # Annual Fee
        if "Annual fee" in text:
            fee_match = re.search(r'\$\d+', text)
            if fee_match:
                annual_fee = fee_match.group()
            else:
                annual_fee = "$0"

        # Welcome Bonus
        if "Welcome bonus" in text or "Intro offer" in text or "Bonus" in text:
            bonus_match = re.search(r'(\$\d{2,}[,\d]*)|(\d{1,3}(?:,\d{3})*)\s*(Points|Miles)', text)
            if bonus_match:
                if bonus_match.group(1):  # Dollar bonus
                    welcome_bonus = f"{bonus_match.group(1)}"
                elif bonus_match.group(2) and bonus_match.group(3):  # Points or miles
                    amount = bonus_match.group(2)
                    unit = bonus_match.group(3).capitalize()
                    welcome_bonus = f"{amount} {unit}"
    #Print output
    print("Name:", card_name)
    if cashback != "No cash back":
        print(cashback)
    if points != "No points":
        print(points + " Miles per dollar")
    print("Annual Fee:", annual_fee)
    print(f"Welcome Bonus: {welcome_bonus}")
    print("")
    
    


Name: Credit One Bank® Platinum X5 Visa®
Cash back: 1% - 5%
Annual Fee: $95
Welcome Bonus: $0

Name: Capital One Quicksilver Cash Rewards Credit Card
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $200

Name: Upgrade Cash Rewards Visa®
Cash back: 1.5%
Annual Fee: $0
Welcome Bonus: $0

Name: Capital One VentureOne Rewards for Good Credit
Points: 1.25x - 5x Miles per dollar
Annual Fee: $0
Welcome Bonus: $0

Name: First Latitude Assent Mastercard® Secured Credit Card
Cash back: 1%
Points: 1.25x - 5x Miles per dollar
Annual Fee: $0
Welcome Bonus: $0

Name: Mission Lane Visa® Credit Card
Cash back: 1%
Annual Fee: $39
Welcome Bonus: $0

Name: Capital One Quicksilver Cash Rewards for Good Credit
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $0

Name: Wells Fargo Active Cash® Card
Cash back: 2%
Annual Fee: $0
Welcome Bonus: $200

Name: Milestone® Mastercard® - With A Higher Credit Limit
Cash back: 2%
Annual Fee: $125
Welcome Bonus: $0

Name: Credit One Bank® Platinum Rewards Visa®
Cas

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

headers = {"User-Agent": "Mozilla/5.0"}

# URLs by credit score tier
pages = {
    "Excellent": "https://www.creditkarma.com/credit-cards/excellent-credit",
    "Good": "https://www.creditkarma.com/credit-cards/good-credit",
    "Fair": "https://www.creditkarma.com/credit-cards/fair-credit",
    "Bad": "https://www.creditkarma.com/credit-cards/bad-credit",
}

all_cards = []

# Loop through each page
for credit_score_label, url in pages.items():
    print(f"Scraping cards for: {credit_score_label} credit...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    credit_cards = soup.find_all("div", class_="jsx-2570701536") 

    for card in credit_cards[:10]:
        card_name_tag = card.find("h2", class_="lh-copy mv0 f4 f3-ns")
        card_name = card_name_tag.text.strip() if card_name_tag else "No name"

        cashback = "No cash back"
        points = "No points"
        apr = "No APR"
        annual_fee = "$0"
        welcome_bonus = "No welcome bonus"

        info_boxes = card.find_all("li")
        for item in info_boxes[:4]:
            text = item.text.strip()

            # Rewards
            if "Cash back" in text or "Miles per dollar" in text or "Rewards rate" in text:
                rewards_match = re.search(
                    r'(\d+(?:\.\d+)?[x%])\s*[-–]\s*(\d+(?:\.\d+)?[x%])|(\d+(?:\.\d+)?[x%])',
                    text, re.IGNORECASE
                )
                if rewards_match:
                    if rewards_match.group(1) and rewards_match.group(2):
                        unit = rewards_match.group(1)[-1]
                        if unit == '%':
                            cashback = f"{rewards_match.group(1)} - {rewards_match.group(2)}"
                        elif unit.lower() == 'x':
                            points = f"{rewards_match.group(1)} - {rewards_match.group(2)}"
                    elif rewards_match.group(3):
                        unit = rewards_match.group(3)[-1]
                        if unit == '%':
                            cashback = f"{rewards_match.group(3)}"
                        elif unit.lower() == 'x':
                            points = f"{rewards_match.group(3)}"

            # APR
            if "APR" in text:
                apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*[-–]\s*(\d+(?:\.\d+)?%))?', text)
                if apr_match:
                    if apr_match.group(2):
                        apr = f"{apr_match.group(1)} - {apr_match.group(2)}"
                    else:
                        apr = f"{apr_match.group(1)}"

            # Annual Fee
            if "Annual fee" in text.lower():
                fee_match = re.search(r'\$\d+', text)
                if fee_match:
                    annual_fee = fee_match.group()

            # Welcome Bonus
            if "welcome bonus" in text.lower() or "intro offer" in text.lower() or "bonus" in text.lower():
                bonus_match = re.search(r'(\$\d{2,}[,\d]*)|(\d{1,3}(?:,\d{3})*)\s*(Points|Miles)', text)
                if bonus_match:
                    if bonus_match.group(1):  # Dollar bonus
                        welcome_bonus = bonus_match.group(1)
                    elif bonus_match.group(2) and bonus_match.group(3):
                        welcome_bonus = f"{bonus_match.group(2)} {bonus_match.group(3).capitalize()}"

        # Store result
        all_cards.append({
            "Name": card_name,
            "Credit Score Tier": credit_score_label,
            "Cashback": cashback,
            "Points": points,
            "APR": apr,
            "Annual Fee": annual_fee,
            "Welcome Bonus": welcome_bonus
        })

# Convert to DataFrame and Save
df = pd.DataFrame(all_cards)
print(df)
#print(all_cards)
df.to_csv("labeled_credit_cards.csv", index=False)



Scraping cards for: Excellent credit...
Scraping cards for: Good credit...
Scraping cards for: Fair credit...
Scraping cards for: Bad credit...
                                                 Name Credit Score Tier  \
0    Capital One Quicksilver Cash Rewards Credit Card         Excellent   
1             Capital One Venture Rewards Credit Card         Excellent   
2                          Upgrade Cash Rewards Visa®         Excellent   
3                      Chase Sapphire Preferred® Card         Excellent   
4                               Citi Simplicity® Card         Excellent   
5                       Citi® Diamond Preferred® Card         Excellent   
6         Bank of America® Travel Rewards credit card         Excellent   
7                              Citi Double Cash® Card         Excellent   
8   Bank of America® Customized Cash Rewards credi...         Excellent   
9          Capital One Savor Cash Rewards Credit Card         Excellent   
10   Capital One Quicksilver Ca

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

headers = {"User-Agent": "Mozilla/5.0"}

# URLs by credit score tier
pages = {
    "Excellent": "https://www.creditkarma.com/credit-cards/excellent-credit",
    "Good": "https://www.creditkarma.com/credit-cards/good-credit",
    "Fair": "https://www.creditkarma.com/credit-cards/fair-credit",
    "Bad": "https://www.creditkarma.com/credit-cards/bad-credit",
}

all_cards = []

# Loop through each page
for credit_score_label, url in pages.items():
    print(f"Scraping cards for: {credit_score_label} credit...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    credit_cards = soup.find_all("div", class_="jsx-2570701536") 

    for card in credit_cards[:10]:
        card_name_tag = card.find("h2", class_="lh-copy mv0 f4 f3-ns")
        card_name = card_name_tag.text.strip() if card_name_tag else "No name"

        cashback = "No cash back"
        points = "No points"
        apr = "No APR"
        annual_fee = "$0"
        welcome_bonus = "No welcome bonus"

        info_boxes = card.find_all("li")
        for item in info_boxes[:4]:
            text = item.text.strip()

            # Rewards
            if "Cash back" in text or "Miles per dollar" in text or "Rewards rate" in text:
                rewards_match = re.search(
                    r'(\d+(?:\.\d+)?[x%])\s*[-–]\s*(\d+(?:\.\d+)?[x%])|(\d+(?:\.\d+)?[x%])',
                    text, re.IGNORECASE
                )
                if rewards_match:
                    if rewards_match.group(1) and rewards_match.group(2):
                        unit = rewards_match.group(1)[-1]
                        if unit == '%':
                            cashback = f"Cash back: {rewards_match.group(1)} - {rewards_match.group(2)}"
                        elif unit.lower() == 'x':
                            points = f"Points: {rewards_match.group(1)} - {rewards_match.group(2)}"
                    elif rewards_match.group(3):
                        unit = rewards_match.group(3)[-1]
                        if unit == '%':
                            cashback = f"Cash back: {rewards_match.group(3)}"
                        elif unit.lower() == 'x':
                            points = f"Points: {rewards_match.group(3)}"

            # APR
            if "APR" in text:
                apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*[-–]\s*(\d+(?:\.\d+)?%))?', text)
                if apr_match:
                    if apr_match.group(2):
                        apr = f"APR: {apr_match.group(1)} - {apr_match.group(2)}"
                    else:
                        apr = f"APR: {apr_match.group(1)}"

            # Annual Fee
            if "Annual fee" in text.lower():
                fee_match = re.search(r'\$\d+', text)
                if fee_match:
                    annual_fee = fee_match.group()

            # Welcome Bonus
            if "welcome bonus" in text.lower() or "intro offer" in text.lower() or "bonus" in text.lower():
                bonus_match = re.search(r'(\$\d{2,}[,\d]*)|(\d{1,3}(?:,\d{3})*)\s*(Points|Miles)', text)
                if bonus_match:
                    if bonus_match.group(1):  # Dollar bonus
                        welcome_bonus = bonus_match.group(1)
                    elif bonus_match.group(2) and bonus_match.group(3):
                        welcome_bonus = f"{bonus_match.group(2)} {bonus_match.group(3).capitalize()}"

       #Print output
        print("Name:", card_name)
        if cashback != "No cash back":
            print(cashback)
        if points != "No points":
            print(points + " Miles per dollar")
        print("Annual Fee:", annual_fee)
        print(f"Welcome Bonus: {welcome_bonus}")
        print("")


Scraping cards for: Excellent credit...
Name: Capital One Quicksilver Cash Rewards Credit Card
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $200

Name: Citi Simplicity® Card
Annual Fee: $0
Welcome Bonus: No welcome bonus

Name: Chase Sapphire Preferred® Card
Points: 1x - 5x Miles per dollar
Annual Fee: $0
Welcome Bonus: 100,000 Points

Name: Capital One Venture Rewards Credit Card
Points: 2x - 5x Miles per dollar
Annual Fee: $0
Welcome Bonus: $915

Name: Upgrade Cash Rewards Visa®
Cash back: 1.5%
Annual Fee: $0
Welcome Bonus: No welcome bonus

Name: Citi Double Cash® Card
Cash back: 1% - 5%
Annual Fee: $0
Welcome Bonus: $200

Name: Bank of America® Customized Cash Rewards credit card
Cash back: 1% - 3%
Annual Fee: $0
Welcome Bonus: $200

Name: Bank of America® Travel Rewards credit card
Points: 1.5x - 3x Miles per dollar
Annual Fee: $0
Welcome Bonus: $250

Name: Citi® Diamond Preferred® Card
Annual Fee: $0
Welcome Bonus: No welcome bonus

Name: Capital One Savor Cash Rewards Cred

In [27]:
dataset = pd.read_csv("labeled_credit_cards.csv")

In [28]:
print(dataset.head())

                                               Name Credit Score Tier  \
0  Capital One Quicksilver Cash Rewards Credit Card         Excellent   
1           Capital One Venture Rewards Credit Card         Excellent   
2                        Upgrade Cash Rewards Visa®         Excellent   
3                    Chase Sapphire Preferred® Card         Excellent   
4                             Citi Simplicity® Card         Excellent   

       Cashback     Points              APR Annual Fee     Welcome Bonus  
0     1.5% - 5%  No points               0%         $0              $200  
1  No cash back    2x - 5x           No APR         $0              $915  
2          1.5%  No points           No APR         $0  No welcome bonus  
3  No cash back    1x - 5x           No APR         $0    100,000 Points  
4  No cash back  No points  18.24% - 28.99%         $0  No welcome bonus  


In [39]:
dataset_clean = dataset.copy()
dataset_clean = pd.get_dummies(dataset_clean, columns=["Credit Score Tier"])
dataset_clean



,Name,Cashback,Points,APR,Annual Fee,Welcome Bonus,Credit Score Tier_Bad,Credit Score Tier_Excellent,Credit Score Tier_Fair,Credit Score Tier_Good
0,Capital One Quicksilver Cash Rewards Credit Card,1.5% - 5%,No points,0%,$0,$200,False,True,False,False
1,Capital One Venture Rewards Credit Card,No cash back,2x - 5x,No APR,$0,$915,False,True,False,False
2,Upgrade Cash Rewards Visa®,1.5%,No points,No APR,$0,No welcome bonus,False,True,False,False
3,Chase Sapphire Preferred® Card,No cash back,1x - 5x,No APR,$0,"100,000 Points",False,True,False,False
4,Citi Simplicity® Card,No cash back,No points,18.24% - 28.99%,$0,No welcome bonus,False,True,False,False
5,Citi® Diamond Preferred® Card,No cash back,No points,17.24% - 27.99%,$0,No welcome bonus,False,True,False,False
6,Bank of America® Travel Rewards credit card,No cash back,1.5x - 3x,No APR,$0,$250,False,True,False,False
7,Citi Double Cash® Card,1% - 5%,No points,No APR,$0,$200,False,True,False,False
8,Bank of America® Customized Cash Rewards credi...,1% - 3%,No points,No APR,$0,$200,False,True,False,False
9,Capital One Savor Cash Rewards Credit Card,1% - 8%,No points,0%,$0,$200,False,True,False,False


In [41]:
def extract_cashback(cashback_str):
    if isinstance(cashback_str, str) and "%" in cashback_str:
        matches = re.findall(r'(\d+(?:\.\d+)?)%', cashback_str)
        if matches:
            return float(max(matches))
    return 0.0

dataset_clean["Cashback"] = dataset_clean["Cashback"].apply(extract_cashback)

In [42]:
def extract_points(points_str):
    if isinstance(points_str, str) and "x" in points_str:
        matches = re.findall(r'(\d+(?:\.\d+)?)x', points_str)
        if matches:
            return float(max(matches))
    return 0.0

dataset_clean["Points"] = dataset_clean["Points"].apply(extract_points)

In [43]:
dataset_clean["Annual Fee"] = dataset_clean["Annual Fee"].apply(lambda x: int(re.sub(r'[^\d]', '', x)) if "$" in str(x) else 0)

In [44]:
def extract_apr(apr_str):
    if isinstance(apr_str, str):
        matches = re.findall(r'(\d+(?:\.\d+)?)%', apr_str)
        if matches:
            nums = [float(num) for num in matches]
            return sum(nums) / len(nums)
    return 0.0

dataset_clean["APR"] = dataset_clean["APR"].apply(extract_apr)

In [45]:
def extract_welcome_bonus(welcome_bonus_str):
    if isinstance(welcome_bonus_str, str):
        if "$" in welcome_bonus_str:
            return int(re.sub(r'[^\d]', '', welcome_bonus_str))
        elif "Points" in welcome_bonus_str or "Miles" in welcome_bonus_str:
            return int(re.sub(r'[^\d]', '', welcome_bonus_str)) // 100
    return 0.0

dataset_clean["Welcome Bonus"] = dataset_clean["Welcome Bonus"].apply(extract_welcome_bonus)

In [46]:
dataset_clean

,Name,Cashback,Points,APR,Annual Fee,Welcome Bonus,Credit Score Tier_Bad,Credit Score Tier_Excellent,Credit Score Tier_Fair,Credit Score Tier_Good
0,Capital One Quicksilver Cash Rewards Credit Card,5.0,0.0,0.000,0,200.0,False,True,False,False
1,Capital One Venture Rewards Credit Card,0.0,5.0,0.000,0,915.0,False,True,False,False
2,Upgrade Cash Rewards Visa®,1.5,0.0,0.000,0,0.0,False,True,False,False
3,Chase Sapphire Preferred® Card,0.0,5.0,0.000,0,1000.0,False,True,False,False
4,Citi Simplicity® Card,0.0,0.0,23.615,0,0.0,False,True,False,False
5,Citi® Diamond Preferred® Card,0.0,0.0,22.615,0,0.0,False,True,False,False
6,Bank of America® Travel Rewards credit card,0.0,3.0,0.000,0,250.0,False,True,False,False
7,Citi Double Cash® Card,5.0,0.0,0.000,0,200.0,False,True,False,False
8,Bank of America® Customized Cash Rewards credi...,3.0,0.0,0.000,0,200.0,False,True,False,False
9,Capital One Savor Cash Rewards Credit Card,8.0,0.0,0.000,0,200.0,False,True,False,False


In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

feature_cols = ["Cashback", "Points", "APR", "Annual Fee", "Welcome Bonus", "Credit Score Tier_Good", "Credit Score Tier_Fair", "Credit Score Tier_Bad"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset_clean[feature_cols])

knn = NearestNeighbors(n_neighbors=5, algorithm='auto')
knn.fit(X_scaled)

def recommend_similar_cards(index, top_n=5):
    distances, indices = knn.kneighbors([X_scaled[index]], n_neighbors=top_n)
    return dataset_clean.iloc[indices[0]][["Name", "Cashback", "Points", "APR", "Annual Fee", "Welcome Bonus"]]

recommendations = recommend_similar_cards(10)
recommendations.reset_index(drop=True, inplace=True)
recommendations

,Name,Cashback,Points,APR,Annual Fee,Welcome Bonus
0,Capital One Quicksilver Cash Rewards Credit Card,5.0,0.0,0.0,0,200.0
1,Capital One Savor Cash Rewards Credit Card,8.0,0.0,0.0,0,200.0
2,Upgrade Cash Rewards Visa®,1.5,0.0,0.0,0,0.0
3,Petal® 2 Visa® Credit Card,10.0,0.0,0.0,0,0.0
4,The New United Gateway℠ Card,0.0,2.0,0.0,0,426.0
